# Hudson plot

## Aim

Create mirrored plots for the joint visualization of different phenotypes/LMM methods

In [ ]:
[global]
# the output directory for generated files
parameter: cwd = path
# GWAS summary statistic file for phenotype/method 1
parameter: sumstats_1= path
#  GWAS summary statistic file for phenotype/method 2
parameter: sumstas_2 =path
# Top Title for the plot
parameter: toptitle=str
# Bottom Title for the plot
parameter: bottomtitle=str
# For cluster jobs, number commands to run per job
parameter: job_size = 1
# Specific number of threads to use
parameter: numThreads = 2
# Column name for BP
parameter: bp = 'POS'
# Column name for p-value
parameter: pval = 'P'
# Column name for SNP
parameter: snp = 'SNP'
# P value use to highlight variants
parameter: highlight_p_top = 0.0
parameter: highlight_p_bottom = 0.0
# SNP list to highlight
parameter: highlight_snp = []
# SNP list to annotate
parameter: annotate_snp = []

In [ ]:
# Hudson plots for two traits
[hudson_plot]
input: sumstats_1, sumstats_2
output: f'{cwd}/{sumstats1:bnn}_{sumstats2:bnn}.hudson.png'
task: trunk_workers = 1, walltime = '10h', mem = '30G', cores = numThreads, tags = f'{step_name}_{_output[0]:bn}'
R: expand= "${ }", stderr = f'{_output[0]:n}.stderr', stdout = f'{_output[0]:n}.stdout'
    library('hudson')
    library('ggplot2')
    library('ggrepel')
    # Import relevant sumstats
    gwas.1 <- read.table(gzfile('${_input1}'), header=T)
    gwas.2 <- read.table(gzfile('${_input2}'), header=T)
  
    # Create the dataframes with useful info
    qqdat1 <- data.frame(SNP=gwas.1$${snp},CHR=gwas.1$CHR,POS=gwas.1$${bp}, pvalue=gwas.1$${pval},length.out=nrow(gwas.1), each=1)
    qqdat2 <- data.frame(SNP=gwas.2$${snp},CHR=gwas.2$CHR,POS=gwas.2$${bp}, pvalue=gwas.2$${pval},length.out=nrow(gwas.2), each=1)
  
    #Create mirrod plot
    gmirror(top=qqdat1, bottom=qqdat2, tline=5e-08, bline=5e-08, 
            toptitle="${toptitle}", bottomtitle = "${bottomtitle}", log10=TRUE,
            highliht_snp = c(${','.join(['"%s"' % x for x in highlight_snp if x is not None])}),
            annotate_snp = c(${','.join(['"%s"' % x for x in annotate_snp if x is not None])}),
            highlight_p = c(${highlight_p_top},${highlight_p_bottom}), highlighter="green",
            file=${_output}, res=300)